In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Configurações iniciais

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 10000)
np.warnings.filterwarnings('ignore')

### Pacotes necessários:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report,\
                            accuracy_score, confusion_matrix, auc
from sklearn.tree import DecisionTreeClassifier,export_graphviz

In [ ]:
def plot_roc_curve(y_true, y_score, figsize=(10,6)):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    plt.figure(figsize=figsize)
    auc_value = roc_auc_score(y_true, y_score)
    plt.plot(fpr, tpr, color='orange', label='ROC curve (area = %0.2f)' % auc_value)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

# Importar a base

In [ ]:
student_mat = pd.read_csv('../input/student-alcohol-consumption/student-mat.csv') 
student_por = pd.read_csv('../input/student-alcohol-consumption/student-por.csv') 


In [ ]:
student_por['count'] = 1

In [ ]:
len(student_por)

In [ ]:
student_por.head()

In [ ]:
student_por.columns

# EDA

* **Variável target**

As variáveis Dalc mede de 1 a 5 o consumo de alcool durante a semana e a variável Walc mede o consumo de alcool no fim de semana. Foi feito a média das duas variáveis resumindo na variável n_consumo_medio, depois foi considerado como "alto consumo" quando esse consumo médio tivesse o valor de 4 ou mais.

In [ ]:
student_por['n_consumo_medio'] = round((student_por['Dalc'] + student_por['Walc'])/2)
student_por['tag_consumo_medio'] = np.where(student_por['n_consumo_medio'] >= 3, 1, 0)

In [ ]:
print(student_por['tag_consumo_medio'].value_counts(normalize=True))
print(student_por['tag_consumo_medio'].value_counts())

In [ ]:
tab = pd.pivot_table(student_por, values='count', columns='tag_consumo_medio', aggfunc='sum').T
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title('Consumo médio')
plt.xticks(tab.index)

In [ ]:
student_por.columns

* #### school

In [ ]:
i = 'school'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(student_por['school'])
student_por['school_dum'] = le.transform(student_por['school'])

dummies_canal = pd.get_dummies(student_por['school_dum']).rename(columns=lambda x: 'school_' + str(x))
student_por = pd.concat([student_por, dummies_canal], axis=1)
student_por.head()

* #### famrel

In [ ]:
i = 'famrel'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T

plt.figure(figsize=(7,5))
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

le = preprocessing.LabelEncoder()
le.fit(student_por['famrel'])
student_por['famrel_dum'] = le.transform(student_por['famrel'])

dummies_canal = pd.get_dummies(student_por['famrel_dum']).rename(columns=lambda x: 'famrel_' + str(x))
student_por = pd.concat([student_por, dummies_canal], axis=1)

freetime

In [ ]:
i = 'freetime'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T

plt.figure(figsize=(7,5))
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

le = preprocessing.LabelEncoder()
le.fit(student_por['freetime'])
student_por['freetime_dum'] = le.transform(student_por['freetime'])

dummies_canal = pd.get_dummies(student_por['freetime_dum']).rename(columns=lambda x: 'freetime_' + str(x))
print(dummies_canal.columns)
student_por = pd.concat([student_por, dummies_canal], axis=1)

goout

In [ ]:
i = 'goout'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T

plt.figure(figsize=(7,5))
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

le = preprocessing.LabelEncoder()
le.fit(student_por['goout'])
student_por['goout_dum'] = le.transform(student_por['goout'])

dummies_canal = pd.get_dummies(student_por['goout_dum']).rename(columns=lambda x: 'goout_' + str(x))
print(dummies_canal.columns)
student_por = pd.concat([student_por, dummies_canal], axis=1)

higher

In [ ]:
i = 'higher'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T

plt.figure(figsize=(7,5))
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

le = preprocessing.LabelEncoder()
le.fit(student_por['higher'])
student_por['higher_dum'] = le.transform(student_por['higher'])

dummies_canal = pd.get_dummies(student_por['higher_dum']).rename(columns=lambda x: 'higher_' + str(x))
print(dummies_canal.columns)
student_por = pd.concat([student_por, dummies_canal], axis=1)

paid

In [ ]:
i = 'paid'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T

plt.figure(figsize=(7,5))
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

le = preprocessing.LabelEncoder()
le.fit(student_por['paid'])
student_por['paid_dum'] = le.transform(student_por['paid'])

dummies_canal = pd.get_dummies(student_por['paid_dum']).rename(columns=lambda x: 'paid_' + str(x))
print(dummies_canal.columns)
student_por = pd.concat([student_por, dummies_canal], axis=1)

internet

In [ ]:
i = 'internet'
tab = pd.pivot_table(student_por, values='count', columns=i, aggfunc='count').T

plt.figure(figsize=(7,5))
plt.bar(tab.index, 'count', width=0.8, bottom=None, align='center', data=tab)
plt.title(i)

le = preprocessing.LabelEncoder()
le.fit(student_por['internet'])
student_por['internet_dum'] = le.transform(student_por['internet'])

dummies_canal = pd.get_dummies(student_por['internet_dum']).rename(columns=lambda x: 'internet_' + str(x))
print(dummies_canal.columns)
student_por = pd.concat([student_por, dummies_canal], axis=1)

# **1. AMOSTRA E BALANCEAMENTO**

In [ ]:
y = student_por['tag_consumo_medio']
x_var = student_por[['school_0','school_1','famrel_0','famrel_1','famrel_2','famrel_3','famrel_4',
                    'goout_0', 'goout_1', 'goout_2', 'goout_3', 'goout_4','internet_0', 'internet_1']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_var, y, test_size=0.30, random_state=27)

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
df_train = pd.concat([X_train, y_train], axis=1)

df_majority_train_group = df_train[df_train['tag_consumo_medio']==0]
df_minority_train_group = df_train[df_train['tag_consumo_medio']==1]
 
df_majority_downsampled_train_group = resample(df_majority_train_group, replace=False, n_samples=len(df_minority_train_group),random_state=1234) 

df_downsampled_train_group = pd.concat([df_majority_downsampled_train_group, df_minority_train_group])
 
df_downsampled_train_group['tag_consumo_medio'].value_counts()

In [ ]:
df_test = pd.concat([X_test, y_test], axis=1)

df_majority_test_nogroup = df_test[df_test['tag_consumo_medio']==0]
df_minority_test_nogroup = df_test[df_test['tag_consumo_medio']==1]
 
df_majority_downsampled_test_nogroup = resample(df_majority_test_nogroup, replace=False, n_samples=len(df_minority_test_nogroup),random_state=1234) 

df_downsampled_test_nogroup = pd.concat([df_majority_downsampled_test_nogroup, df_minority_test_nogroup])
 
df_downsampled_test_nogroup['tag_consumo_medio'].value_counts()

Variáveis selecionadas

In [ ]:
df_downsampled_train_group.columns

In [ ]:
x_treino = df_downsampled_train_group[['school_0','school_1','famrel_0','famrel_1','famrel_2','famrel_3','famrel_4',
                    'goout_0', 'goout_1', 'goout_2', 'goout_3', 'goout_4','internet_0', 'internet_1']]
y_treino = df_downsampled_train_group['tag_consumo_medio']

In [ ]:
x_teste = df_downsampled_test_nogroup[['school_0','school_1','famrel_0','famrel_1','famrel_2','famrel_3','famrel_4',
                    'goout_0', 'goout_1', 'goout_2', 'goout_3', 'goout_4','internet_0', 'internet_1']]
y_teste = df_downsampled_test_nogroup['tag_consumo_medio']

**1. SELEÇÃO DE VARIÁVEIS**

In [ ]:
reg = LassoCV()
reg.fit(x_var, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(x_var,y))
coef = pd.Series(reg.coef_, index = x_var.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Seleção de variáveis usando Regressão LASSO - Variáveis dummies")

In [ ]:
var_retirar = ['internet_0', 'internet_1']
x_treino.drop(columns=var_retirar,inplace=True)
x_teste.drop(columns=var_retirar,inplace=True)

# **MODELAGEM**

### Regressão Logística

Treino

In [ ]:
len(x_treino.columns)

In [ ]:
modelo_logistic = LogisticRegression(penalty='none', solver='newton-cg')
modelo_logistic.fit(x_treino, y_treino)
y_treino_pred = modelo_logistic.predict(x_treino)

In [ ]:
confusion_matrix(y_treino, y_treino_pred)

In [ ]:
acuracia_logistica_train = accuracy_score(y_treino, y_treino_pred)
print('O modelo obteve %0.4f de acurácia.' % acuracia_logistica_train)

In [ ]:
roc_logistic_train = roc_auc_score(y_treino, y_treino_pred)
print('AUC: %0.2f' % roc_auc_score(y_treino, y_treino_pred))
plot_roc_curve(y_treino, y_treino_pred)

Teste

In [ ]:
modelo_logistic = LogisticRegression(penalty='none', solver='newton-cg')
modelo_logistic.fit(x_teste, y_teste)
y_teste_pred = modelo_logistic.predict(x_teste)

In [ ]:
confusion_matrix(y_teste, y_teste_pred)

In [ ]:
acuracia_logistica_train = accuracy_score(y_teste, y_teste_pred)
print('O modelo obteve %0.4f de acurácia.' % acuracia_logistica_train)

In [ ]:
roc_logistic_train = roc_auc_score(y_teste, y_teste_pred)
print('AUC: %0.2f' % roc_auc_score(y_teste, y_teste_pred))
plot_roc_curve(y_teste, y_teste_pred)

### Árvore de Decisão

Treino

In [ ]:
len(y_teste)

In [ ]:
modelo_tree = DecisionTreeClassifier()
modelo_tree = modelo_tree.fit(x_treino,y_treino)
y_treino_pred = modelo_tree.predict(x_treino)

In [ ]:
confusion_matrix(y_treino, y_treino_pred)

In [ ]:
acuracia_logistica_train = accuracy_score(y_treino, y_treino_pred)
print('O modelo obteve %0.4f de acurácia.' % acuracia_logistica_train)

In [ ]:
roc_logistic_train = roc_auc_score(y_treino, y_treino_pred)
print('AUC: %0.2f' % roc_auc_score(y_treino, y_treino_pred))
plot_roc_curve(y_treino, y_treino_pred)

In [ ]:
modelo_tree.feature_importances_

In [ ]:
for feature,importancia in zip(x_treino.columns,modelo_tree.feature_importances_):
    print("{}:{}".format(feature, importancia))

Teste

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(x_teste,y_teste)
y_teste_pred = clf.predict(x_teste)

confusion_matrix(y_teste, y_teste_pred)

In [ ]:
acuracia_logistica_train = accuracy_score(y_teste, y_teste_pred)
print('O modelo obteve %0.4f de acurácia.' % acuracia_logistica_train)

In [ ]:
roc_logistic_train = roc_auc_score(y_teste, y_teste_pred)
print('AUC: %0.2f' % roc_auc_score(y_teste, y_teste_pred))
plot_roc_curve(y_teste, y_teste_pred)